<a href="https://colab.research.google.com/github/essteer/data-science/blob/main/src/neural_nets/tensorflow_ann_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow ANN - Titanic Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
df = pd.read_csv("titanic.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df["Ticket"].nunique()

681

In [ ]:
df["Cabin"].nunique()

147

In [ ]:
# Drop arbitrary values
df = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

In [ ]:
df.iloc[[0, -1]]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.25,S
890,0,3,male,32.0,0,0,7.75,Q


In [ ]:
df["Survived"].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [ ]:
df["Embarked"].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [ ]:
# Replace NaN values with most common value
# Only 2/891 values are missing, so impute median before train test split
df["Embarked"].replace(np.nan, "S", inplace=True)

## Train test split

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, :1].values

In [ ]:
X.shape

(891, 7)

In [ ]:
y.shape

(891, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [ ]:
X_train.shape

(712, 7)

In [ ]:
X_test.shape

(179, 7)

## Clean the data

In [ ]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [ ]:
# Calculate mean train age (exlude NaNs)
train_age = X_train[:, 2].astype(float)
mean_train_age = np.nanmean(train_age[~np.isnan(train_age)])

In [ ]:
# Replace NaN values with mean using nan_to_num
train_age = np.nan_to_num(train_age, nan=mean_train_age)

In [ ]:
X_train[:, 2] = train_age

In [ ]:
# Calculate mean test age (exclude NaNs)
test_age = X_test[:, 2].astype(float)
mean_test_age = np.nanmean(test_age[~np.isnan(test_age)])

In [ ]:
# Replace NaN values with mean using nan_to_num
test_age = np.nan_to_num(test_age, nan=mean_test_age)

In [ ]:
X_test[:, 2] = test_age

## Transform features

In [ ]:
# Encode data - don't scale until train-test split
labelencoder_Sex = LabelEncoder()
X_train[:, 1] = labelencoder_Sex.fit_transform(X_train[:, 1])
X_test[:, 1] = labelencoder_Sex.transform(X_test[:, 1])
labelencoder_Embarked = LabelEncoder()
X_train[:, 6] = labelencoder_Embarked.fit_transform(X_train[:, 6])
X_test[:, 6] = labelencoder_Embarked.transform(X_test[:, 6])

## Scale features

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train

array([[ 8.30566141e-01,  7.35612358e-01,  1.14779706e+00, ...,
        -4.52479538e-01, -5.12175428e-01,  5.83683870e-01],
       [-1.55877969e+00,  7.35612358e-01,  1.30196252e+00, ...,
        -4.52479538e-01,  3.23200846e-02,  5.83683870e-01],
       [-3.64106773e-01,  7.35612358e-01,  4.92593862e-01, ...,
         1.98816767e+00, -1.29157469e-01,  5.83683870e-01],
       ...,
       [ 8.30566141e-01,  7.35612358e-01, -7.79271170e-01, ...,
        -4.52479538e-01, -4.93049696e-01,  5.83683870e-01],
       [ 8.30566141e-01,  7.35612358e-01,  1.07071433e+00, ...,
        -4.52479538e-01, -4.90533152e-01,  5.83683870e-01],
       [ 8.30566141e-01, -1.35941164e+00, -8.21558600e-16, ...,
        -4.52479538e-01, -5.07142341e-01, -1.95810766e+00]])

## ANN

In [ ]:
# Initialise the ANN
classifier = Sequential()

Rule of thumb for determining the number of hidden unit:
(no. input dimensions + no. output dimensions) / 2

7 independent variables + 1 dependent variable
(7 + 1) / 2 = 4.

In [ ]:
# Input layer and first hidden layer
classifier.add(Dense(units=14, kernel_initializer="uniform", activation="relu", input_dim=7))
# Second hidden layer
classifier.add(Dense(units=14, kernel_initializer="uniform", activation="relu"))
# Third  hidden layer
classifier.add(Dense(units=4, kernel_initializer="uniform", activation="relu"))
# Output layer - sigmoid for binary classification
classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

In [ ]:
X_train.shape

(712, 7)

This is a binary classification task, so the NN will use the binary_crossentropy loss function.

Options for other multi-class classification tasks include categorical_crossentropy and sparsecategorical_crossentropy.

In [ ]:
# Compile the ANN
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

The training set has 712 observations.

- With batch_size=4, epochs=178
- With batch_size=8, epochs=89

In [ ]:
# Fit the ANN to the training set
classifier.fit(X_train, y_train, batch_size=8, epochs=100)

Epoch 1/100
89/89 [==============================] - 1s 2ms/step - loss: 0.6894 - accuracy: 0.6096
Epoch 2/100
89/89 [==============================] - 0s 2ms/step - loss: 0.6373 - accuracy: 0.6868
Epoch 3/100
89/89 [==============================] - 1s 17ms/step - loss: 0.4987 - accuracy: 0.7949
Epoch 4/100
89/89 [==============================] - 1s 14ms/step - loss: 0.4561 - accuracy: 0.7963
Epoch 5/100
89/89 [==============================] - 1s 6ms/step - loss: 0.4464 - accuracy: 0.7992
Epoch 6/100
89/89 [==============================] - 0s 4ms/step - loss: 0.4394 - accuracy: 0.7949
Epoch 7/100
89/89 [==============================] - 0s 4ms/step - loss: 0.4367 - accuracy: 0.7935
Epoch 8/100
89/89 [==============================] - 0s 4ms/step - loss: 0.4340 - accuracy: 0.8118
Epoch 9/100
89/89 [==============================] - 0s 4ms/step - loss: 0.4322 - accuracy: 0.8090
Epoch 10/100
89/89 [==============================] - 0s 4ms/step - loss: 0.4318 - accuracy: 0.8048
Epoch 1

## Test the model

In [ ]:
y_pred = classifier.predict(X_test)

6/6 [==============================] - 0s 3ms/step


In [ ]:
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(f"Test set confusion matrix:\n{cm}")

Test set confusion matrix:
[[103   9]
 [ 18  49]]


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {accuracy:.2%}")

Test set accuracy: 84.92%


In [ ]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.85      0.92      0.88       112
           1       0.84      0.73      0.78        67

    accuracy                           0.85       179
   macro avg       0.85      0.83      0.83       179
weighted avg       0.85      0.85      0.85       179

